## Imports

In [1]:
import os

In [2]:
from gensim.models.wrappers.fasttext import FastText

## Config

In [3]:
feature_list_id = 'wmd'

In [4]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep

## Read Data

In [5]:
df_questions_train = pd.read_csv(data_folder + 'train.csv').fillna('')
df_questions_test = pd.read_csv(data_folder + 'test.csv').fillna('')

In [6]:
question_tokens_train = load_json(preproc_data_folder + 'question_tokens_train.json')
question_tokens_test = load_json(preproc_data_folder + 'question_tokens_test.json')

In [7]:
embedding_model = FastText.load_word2vec_format(aux_data_folder + 'quora_filtered.vec')

## Build Features

In [8]:
def build_features(questions_tokenized, embedding_model):
    num_pairs = len(questions_tokenized)
    word_vector_dim = len(embedding_model['apple'])
    num_features = 1
    
    X = np.zeros((num_pairs, num_features), dtype=float)
    
    for index, pair in progressbar(enumerate(questions_tokenized), size=num_pairs):
        # Word Mover's Distance between the documents
        X[index, 0] = embedding_model.wmdistance(pair['question1'], pair['question2'])
    
    return X

In [9]:
X_train = build_features(question_tokens_train, embedding_model)

In [10]:
save(X_train, features_data_folder + f'X_train_{feature_list_id}.pickle')

In [11]:
X_test = build_features(question_tokens_test, embedding_model)

In [12]:
save(X_test, features_data_folder + f'X_test_{feature_list_id}.pickle')